# MASONGO UMAR
1800738510         2018/HD05/1967U
## MASTER OF SCIENCE IN COMPUTER SCIENCE
COMPUTER VISION PROJECT EXAM

In [2]:
from collections import deque
from datetime import datetime
import os
import pickle
import time
import cv2
import numpy as np
from scipy.ndimage.measurements import label
%run hogDescriptor.ipynb
%run slideWindow.ipynb
%run processTrain.ipynb

Building file list...
160 positive files and 54 negative files found.

Converting images to BGR color space and extracting HOG, color histogram, spatial features from channel(s) 0, 1, 2.

Features extracted from 214 files in 53.1 seconds

Scaling features.

Shuffling samples into training, cross-validation, and test sets.

120 samples in positive training set.
32 samples in positive cross-validation set.
8 samples in positive test set.
160 total positive samples.

40 samples in negative training set.
11 samples in negative cross-validation set.
3 samples in negative test set.
54 total negative samples.

Loading sample data.
Training classifier...


/home/umar/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Classifier trained in 2.3 s.

Val set false negatives: 1 / 32 (96.9% accuracy)
Val set false positives: 0 / 11 (100.000% accuracy)
Val set total misclassifications: 1 / 43 (97.674% accuracy)

Augmenting training set with misclassified validation samples and retraining classifier.

Test set false negatives: 0 / 8 (1e+02% accuracy)
Test set false positives: 0 / 3 (100.000% accuracy)
Test set total misclassifications: 0 / 11 (100.000% accuracy)


In [4]:
class Detector:

    # This class is used to detect objects(Cassava) in a video stream
    # A pretrained SVM is loaded from trainSVM file and used
    
    descriptor = Descriptor()
     
    def __init__(self, init_size=(64,64), x_overlap=0.0463, y_step=0.0261,
            x_range=(0, 1.0), y_range=(0, 1.0), scale=1.0):
        
        # Input arguments that set the sliding window parameters as defined in the slidingWindow file

        self.init_size = init_size
        self.x_overlap = x_overlap
        self.y_step = y_step
        self.x_range = x_range
        self.y_range = y_range
        self.scale = scale
        self.windows = None
        
    def loadClassifier(self, filepath=None, classifier_data=None):
        
        # Load a classifier trained by trainSVM function in processTrain.ipynb Either a dict
        # (classifier_data) or pickled file (filepath) can be supplied.
    
        if filepath is not None:
            filepath = os.path.abspath(filepath)
            if not os.path.isfile(filepath):
                raise FileNotFoundError("File " + filepath + " does not exist.")
            classifier_data = pickle.load(open(filepath, "rb"))
        else:
            classifier_data = classifier_data

        if classifier_data is None:
            raise ValueError("Invalid classifier data supplied.")

        self.classifier = classifier_data["classifier"]
        self.scaler = classifier_data["scaler"]
        self.cv_color_const = classifier_data["cv_color_const"]
        self.channels = classifier_data["channels"]
        self.descriptor = Descriptor(
                hog_features=classifier_data["hog_features"],
                hist_features=classifier_data["hist_features"],
                spatial_features=classifier_data["spatial_features"],
                hog_lib=classifier_data["hog_lib"],
                size=classifier_data["size"],
                hog_bins=classifier_data["hog_bins"],
                pix_per_cell=classifier_data["pix_per_cell"],
                cells_per_block=classifier_data["cells_per_block"],
                block_stride=classifier_data["block_stride"],
                block_norm=classifier_data["block_norm"],
                transform_sqrt=classifier_data["transform_sqrt"],
                signed_gradient=classifier_data["signed_gradient"],
                hist_bins=classifier_data["hist_bins"],
                spatial_size=classifier_data["spatial_size"])
        
        return self
    
    def classify(self, image):
        
        # Classify windows of an image as "positive" (Contains banana crop) or "negative" (contains other crops)
        # Return a list of positively classified windows.
        
        if self.cv_color_const > -1:
            image = cv2.cvtColor(image, self.cv_color_const)

        if len(image.shape) > 2:
            image = image[:, :, self.channels]
        else:
            image = image[:, :, np.newaxis]

        #feature_vectors = [Descriptor.getFeatureVector(descriptor,
                #image[y_upper:y_lower, x_upper:x_lower, :])
            #for (x_upper, y_upper, x_lower, y_lower) in self.windows]
        
        feature_vectors = [descriptor.getFeatureVector(
                image[y_upper:y_lower, x_upper:x_lower, :])
            for (x_upper, y_upper, x_lower, y_lower) in self.windows]
            
        # feature_vectors = self.to_matrix(feature_vectors, 2)

        # Scale feature vectors, predict, and return predictions
        feature_vectors = self.scaler.transform(feature_vectors)
        predictions = self.classifier.predict(feature_vectors)
        
        return [self.windows[ind] for ind in np.argwhere(predictions == 1)[:,0]]
    
    def crop_detection(self, video_capture=None, num_frames=9, threshold=120,
            min_bbox=None, show_video=True, draw_heatmap=True,
            draw_heatmap_size=0.2, write=False, write_fps=24):
        
        cap = video_capture
        if not cap.isOpened():
            raise RuntimeError("Error opening VideoCapture.")
        (grabbed, frame) = cap.read()
        (h, w) = frame.shape[:2]

        # Store coordinates of all windows to be checked at every frame.
        self.windows = slidingWindow((w, h), init_size=self.init_size,
                x_overlap=self.x_overlap, y_step=self.y_step,
                x_range=self.x_range, y_range=self.y_range, scale=self.scale)

        if min_bbox is None:
            min_bbox = (int(0.02 * w), int(0.02 * h))

        # Heatmap inset size.
        inset_size = (int(draw_heatmap_size * w), int(draw_heatmap_size * h))

        if write:
            vidFilename = datetime.now().strftime("%Y%m%d%H%M") + ".avi"
            fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
            writer = cv2.VideoWriter(vidFilename, fourcc, write_fps, (w, h))

        # Compute the heatmap for each frame and store in current_heatmap.
        # Store the last num_frames heatmaps in deque last_N_frames. At each
        # frame, sum in the deque to compute summed_heatmap. After
        # thresholding, label blobs in summed_heatmap with
        # scipy.ndimage.measurements.label and store in heatmap_labels.
        current_heatmap= np.zeros((frame.shape[:2]), dtype=int)
        summed_heatmap = np.zeros_like(current_heatmap, dtype=int)
        last_N_frames = deque(maxlen=num_frames)
        heatmap_labels = np.zeros_like(current_heatmap, dtype=int)

        # Weights for the frames in last_N_frames for producing summed_heatmap.
        # Recent frames are weighted more heavily than older frames.
        weights = np.linspace(1 / (num_frames + 1), 1, num_frames)
        while True:
            (grabbed, frame) = cap.read()
            if not grabbed:
                break

            current_heatmap[:] = 0
            summed_heatmap[:] = 0
            for (x_upper, y_upper, x_lower, y_lower) in self.classify(frame):
                current_heatmap[y_upper:y_lower, x_upper:x_lower] += 10

            last_N_frames.append(current_heatmap)
            for i, heatmap in enumerate(last_N_frames):
                cv2.add(summed_heatmap, (weights[i] * heatmap).astype(np.uint8),
                    dst=summed_heatmap)

            # Apply blur and/or dilate to the heatmap.
            #cv2.GaussianBlur(summed_heatmap, (5,5), 0, dst=summed_heatmap)
            cv2.dilate(summed_heatmap, np.ones((7,7), dtype=np.uint8),
                dst=summed_heatmap)

            if draw_heatmap:
                inset = cv2.resize(summed_heatmap, inset_size,
                    interpolation=cv2.INTER_AREA)
                inset = cv2.cvtColor(inset, cv2.COLOR_GRAY2BGR)
                frame[:inset_size[1], :inset_size[0], :] = inset

            # Ignore heatmap pixels below threshold.
            summed_heatmap[summed_heatmap <= threshold] = 0

            # Label remaining blobs with scipy.ndimage.measurements.label.
            num_objects = label(summed_heatmap, output=heatmap_labels)

            # Determine the largest bounding box around each object.
            for obj in range(1, num_objects + 1):
                (Y_coords, X_coords) = np.nonzero(heatmap_labels == obj)
                x_upper, y_upper = min(X_coords), min(Y_coords)
                x_lower, y_lower = max(X_coords), max(Y_coords)

                # Only draw box if object is larger than min bbox size.
                if (x_lower - x_upper > min_bbox[0]
                        and y_lower - y_upper > min_bbox[1]):
                    cv2.rectangle(frame, (x_upper, y_upper), (x_lower, y_lower),
                        (0, 255, 0), 6)

            if write:
                writer.write(frame)

            if show_video:
                cv2.imshow("Detection", frame)
                cv2.waitKey(1)

        cap.release()

        if write:
            writer.release()
    

In [5]:
# Instantiate the banana detector
banana_detector = Detector()

In [6]:
# Set desired sliding window parameters
banana_detector = Detector(init_size=(64, 64), x_overlap=0.0463, y_step=0.0261, x_range=(0, 1.0), y_range=(0, 1.0), scale=1.0)

In [7]:
# Load classifier from dict
banana_detector.loadClassifier(filepath=None, classifier_data=classifier_data)

In [9]:
# Run the detector on a video by creating an OpenCV VideoCapture object and
# supplying it to Detector.detectVideo()

cap = cv2.VideoCapture("./hog_image/Garden_Video.mp4")